excel-to-sqlite-python
======================



## README



This is a short example to show how to

1.  Write SQLite data to CSV files
2.  Import CSV files into Excel workbook ([tinyurl.com/foods-csv](https://tinyurl.com/foods-csv))
3.  Read Excel data into a Python dataframe
4.  Create an SQLite database for the data
5.  Insert the data into the SQLite database

This was written for neo-Pythonistas who already know and understand
SQLite database design and manipulation.

Source: "Turn Your Excel Workbook Into a SQLite Database" by
S.A. Adams (May 18, 2020). URL: [towardsdatascience.com](https://towardsdatascience.com/turn-your-excel-workbook-into-a-sqlite-database-bc6d4fd206aa).



## Write SQLite data to a CSV file



-   Instead of the article data, I am using a very simple test file:
    
        id,value
        1,"Ms. Jane Robinson"
        2,"Mr. Carl Robinson"
        3,"Mr. Edward Jones"



## Turn the CSV file into an Excel file



-   You either need Microsoft Excel for this, or a free clone like
    LibreOffice spreadsheet, or Google Docs.

-   I'm using Google Docs to `import` the CSV files:
    
    ![img](../img/csv_to_excel.png)

-   The result:
    
    ![img](../img/csv_to_excel3.png)

-   This is the file that we'll read into a Python `Data.Frame`. You can
    find the online file here for download:
    [tinyurl.com/excel-to-sqlite-csv](https://tinyurl.com/excel-to-sqlite-csv)



## Read Excel data into a Python dataframe using `pd.read_excel`



-   Python is an all-purpose high-level programming language used much
    in data science in machine learning but also useful for general
    scripting and automating of tasks.

-   For data science, the `pandas` package is especially useful: as you
    can read in the online [documentation](https://pandas.pydata.org/pandas-docs/stable/index.html), `pandas` provides data analysis
    tools to Python.

-   If you do this in an interactive DataCamp DataLab or Google Colab
    notebook, `pandas` will already be installed and you only have to load
    it<a href="#1">[1]</a>.

-   To use `pandas,` you have to `import` the library:



In [1]:
import pandas as pd

-   Now, you have access to `pandas` functions, e.g. `pd.read_excel`:
    
        help(pd.read_excel)

-   Here's the top of the `help` output:
    
        read_excel(io, sheet_name: 'str | ... header: 'int | Sequence[int] | None' = 0, ...]'
        
            Read an Excel file into a ``pandas`` ``DataFrame``.
        
            Supports `xls`, `xlsx`, `xlsm`, `xlsb`, `odf`, `ods` and `odt` file extensions
            read from a local filesystem or URL. Supports an option to read
            a single sheet or a list of sheets.
        
            [...]
        
            See Also
            --------
            DataFrame.to_excel : Write DataFrame to an Excel file.
            DataFrame.to_csv : Write DataFrame to a comma-separated values (csv) file.
            read_csv : Read a comma-separated values (csv) file into DataFrame.
            read_fwf : Read a table of fixed-width formatted lines into DataFrame.

-   You can find out much more about `read_excel` in the online
    [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html). As you can see in the `help`, the function only has one
    mandatory argument `io`, which can be a URL string or an Excel file
    name (in quotes).

-   The `header` parameter is 0 by default - we're OK because we got one.

-   URL import, especially from Google Docs, does not always work: to be
    on the safe side, I've downloaded the Excel file as `test.xlsx`:
    
    ![img](../img/xlsx2.png)

-   We import the data into a `Data.Frame` named `df`:



In [1]:
df = pd.read_excel('test.xlsx')
print(df.head())

:    id              value
  : 0   1  Ms. Jane Robinson
  : 1   2  Mr. Carl Robinson
  : 2   3   Mr. Edward Jones

-   You see that there's an extra column for the row index starting
    at 0. The `pandas` function `info` provides overall information:



In [1]:
print(df.info())

#+begin_example
  <class 'pandas.core.frame.DataFrame'>
  RangeIndex: 3 entries, 0 to 2
  Data columns (total 2 columns):
   #   Column  Non-Null Count  Dtype
  ---  ------  --------------  -----
   0   id      3 non-null      int64
   1   value   3 non-null      object
  dtypes: int64(1), object(1)
  memory usage: 176.0+ bytes
  None
  #+end_example

## Create SQLite database and put the data into it



![img](../img/sqlite3_python.png "Source: pynative.com/python-sqlite/")

-   We're now going to create a `test.db` SQLite database using Python's
    `sqlite3` package, which needs to be imported (or installed):



In [1]:
import sqlite3

-   As you can read in the [documentation](https://docs.python.org/3/library/sqlite3.html), `sqlite3` is a database
    interface for SQLite databases: it allows you to submit SQLite
    commands from within a Python script. There is also a [tutorial](https://docs.python.org/3/library/sqlite3.html#sqlite3-tutorial).

-   The image illustrates how the Python modulde `sqlite3` works:
    1.  You run SQLite operations (like `SELECT`) in Python and results are
        returned to the Python console.
    2.  The `sqlite3` module executes queries on the SQLite database, and
        fetches results from the SQLite database.
    3.  To establish data transfer between database and Python script,
        there is an Application Programming Interface (API), [PEP 249](https://peps.python.org/pep-0249/).

-   The steps to hitching SQLite to Python are as follows:
    1.  With `sqlite3.connect`, initiate a new SQLite database connection
        object `db_conn`, which creates an (empty) database `test.db`.
    2.  Run a `cursor` object on the connection. This object lets us
        `execute` SQLite data definition commands like `CREATE TABLE`.
    3.  Run the `pandas` function `to_sql` on a `DataFrame` to `INSERT` data
        into an SQLite table.
    4.  To execute SQLite queries on a given database, run `SELECT`
        commands on the tables using the `pandas` function `read_sql`.



## Initiate a database connection creating an empty database (`sqlite3.connect`)



-   Remove the `test.db` database if it already exists:



In [1]:
import os
os.system("rm ../data/test.db")

-   Creating a connection object also creates an (empty) database:



In [1]:
db_conn = sqlite3.connect("../data/test.db")

-   Type of object:



In [1]:
print(type(db_conn))

: <class 'sqlite3.Connection'>

-   Check the empty database (`os.system` executes OS shell commands):



In [1]:
os.system("ls -l ../data/test.db")

: -rw-r--r-- 1 marcus marcus 0 May 22 11:21 ../data/test.db

## Run data definition commands on the database to create tables (`db_conn.cursor`)



-   Data definition means that we need to devise a schema.

-   We want a very simple database schema:
    
        CREATE TABLE test (id INTEGER PRIMARY KEY,
                           value TEXT);

-   The `DataFrame` objects where we stored the data, are already aligned
    with this database design (apart from the bridge table
    `foods_episodes`):



In [1]:
print(df.columns)

: Index(['id', 'value'], dtype='object')

### SQLite database reference `cursor`



-   This is the database design that we're now going to build using the
    `Cursor` object `db_conn.cursor` - a reference pointing at the database:



In [1]:
c = db_conn.cursor()
print(type(c))

: <class 'sqlite3.Cursor'>

-   You can get `help` on this object directly, or check the
    [documentation](https://docs.python.org/3/library/sqlite3.html#sqlite3.Cursor)<a href="#2">[2]</a>



In [1]:
help(db_conn.cursor())

#+begin_example
  Help on Cursor in module sqlite3 object:

  class Cursor(builtins.object)
   |  SQLite database cursor class.
   |
   |  Methods defined here:
   |
   |  __init__(self, /, *args, **kwargs)
   |      Initialize self.  See help(type(self)) for accurate signature.
   |
   |  __iter__(self, /)
   |      Implement iter(self).
   |
   |  __next__(self, /)
   |      Implement next(self).
   |
   |  close(self, /)
   |      Closes the cursor.
   |
   |  execute(self, sql, parameters=(), /)
   |      Executes an SQL statement.
   |
   |  executemany(self, sql, seq_of_parameters, /)
   |      Repeatedly executes an SQL statement.
   |
   |  executescript(self, sql_script, /)
   |      Executes multiple SQL statements at once.
   |
   |  fetchall(self, /)
   |      Fetches all rows from the resultset.
   |
   |  fetchmany(self, /, size=1)
   |      Fetches several rows from the resultset.
   |
   |      size
   |        The default value is set by the Cursor.arraysize attribute.

-   Now create the table `test` using the reference to `test.db`:



In [1]:
c.execute(
    """
    CREATE TABLE
       IF NOT EXISTS
       test (
       id INTEGER PRIMARY KEY,
       value TEXT
       );
    """
)

-   Check that the table was created:



In [1]:
tab = c.execute("SELECT name FROM sqlite_master")
print(tab.fetchone())
os.system("ls -l ../data/test.db")

: ('test',)
  : -rw-r--r-- 1 marcus marcus 8192 May 22 11:22 ../data/test.db

-   The query returns a tuple containing the table's name `test` - still
    empty except for the table definition.



## Insert data from the `DataFrame` into database tables (`pd.to_sql`)



-   This command transfers the content of `df` to the `test` table in our
    database.



In [1]:
df.to_sql('test',  # target table
          db_conn, # database connection
          if_exists='append', # append data if table exists
          index=False)  # do not add DataFrame index as a table column

## Run queries on the database tables (`pd.read_sql`)



-   To run queries on the data, we use `pandas` function `read_sql`. The
    first argument is the command, the second the database connection:



In [1]:
query = pd.read_sql("SELECT * FROM test", db_conn)
print(query)

:    id              value
  : 0   1  Ms. Jane Robinson
  : 1   2  Mr. Carl Robinson
  : 2   3   Mr. Edward Jones

-   The first column is not a table column but the index column of the
    output `DataFrame`:



In [1]:
print(query.info())

#+begin_example
  <class 'pandas.core.frame.DataFrame'>
  RangeIndex: 3 entries, 0 to 2
  Data columns (total 2 columns):
   #   Column  Non-Null Count  Dtype 
  ---  ------  --------------  ----- 
   0   id      3 non-null      int64 
   1   value   3 non-null      object
  dtypes: int64(1), object(1)
  memory usage: 176.0+ bytes
  None
  #+end_example

-   The `read_sql` function is a wrapper around two other functions from
    the `SQLAlchemy` toolkit - if you want to get more deeply into writing
    Python scripts for database access, check out the [documentation](https://www.sqlalchemy.org/).



## Footnotes



<p id="2">[2] The cursor may appear like a pointless abstraction to you - why
not just use the connection object? The reason is encapsulation of
SQLite commands - the connection manages the connection to the
database, while the cursor contains methods to execute SQLite
commands. The cursor also maintains the state of the current query,
which is critical for fetching data in chunks and adds efficiency.
<p id="1">[1] You do not need a fancy setup with the `conda` platform if you use
an interactive ('Jupyter') notebook installation in the cloud. If
you're using Emacs (which is what I do), you're also set
(locally). What I've done is write all of this as a literate program
in Emacs, which I will then render as an IPython notebook
(`excel_to_sqlite.ipynb`), upload to DataLab and share with you.

